In [14]:
import numpy as np 
import pandas as pd
import re
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from scipy import special
from tqdm import tqdm
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.impute import SimpleImputer 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score, classification_report, accuracy_score, roc_curve
import matplotlib.pyplot as plt
import optuna
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMClassifier, LGBMRegressor
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.feature_selection import RFE
from boruta import BorutaPy

warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:20,.2f}'.format)

In [2]:
X_train = pd.read_csv('/Users/sakshamjain/Desktop/Projects/JAIN-WIN/X_train_corr.csv')
y_train = pd.read_csv('/Users/sakshamjain/Desktop/Projects/JAIN-WIN/y_train_large.csv').squeeze()
X_test = pd.read_csv('/Users/sakshamjain/Desktop/Projects/JAIN-WIN/X_test_corr.csv')
y_test = pd.read_csv('/Users/sakshamjain/Desktop/Projects/JAIN-WIN/y_test_large.csv').squeeze()

In [3]:
X_train.head()

,age,bmi,elective_surgery,gender,height,icu_id,pre_icu_los_days,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,icu_id_counts,age_counts,nan_counts,sq_age,sq_bmi,bmi_age,weight_age,comorbidity_score,diabete_risk,gcs_sum,apache_2_diagnosis_type,bmi_type,height_type,weight_type,age_type,gcs_sum_type,apache_3j_diagnosis_split1,apache_2_diagnosis_split1,profile,diff_bmi,bilirubin_d1_h1_max_eq,bilirubin_d1_h1_min_eq,bilirubin_h1_value_range,bilirubin_h1_zero_range,bilirubin_h1_mean,bilirubin_h1_std,bilirubin_h1_max_apache_3j_diagnosis_mean,bilirubin_h1_max_apache_3j_diagnosis_diff,bilirubin_h1_max_apache_3j_diagnosis_std,bilirubin_h1_max_apache_3j_diagnosis_norm_std,bilirubin_h1_max_apache_3j_diagnosis_rank,bilirubin_h1_max_apache_3j_diagnosis_count,bilirubin_h1_max_apache_3j_diagnosis_norm_rank,bilirubin_h1_max_apache_3j_diagnosis_skew,bilirubin_h1_min_apache_3j_diagnosis_mean,bilirubin_h1_min_apache_3j_diagnosis_diff,bilirubin_h1_min_apache_3j_diagnosis_std,bilirubin_h1_min_apache_3j_diagnosis_norm_std,bilirubin_h1_min_apache_3j_diagnosis_rank,bilirubin_h1_min_apache_3j_diagnosis_norm_rank,bilirubin_h1_min_apache_3j_diagnosis_skew,bilirubin_h1_max_profile_mean,bilirubin_h1_max_profile_diff,bilirubin_h1_max_profile_std,bilirubin_h1_max_profile_norm_std,bilirubin_h1_max_profile_rank,bilirubin_h1_max_profile_count,bilirubin_h1_max_profile_norm_ran

In [4]:
print("Training LightGBM model...")
model = lgb.LGBMClassifier( random_state=69, n_jobs=-1, force_col_wise=True )
# Train the initial LightGBM model
model.fit(X_train, y_train)

# Get feature importance
feature_importance = model.feature_importances_
feature_names = X_train.columns

Training LightGBM model...
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 19677, number of negative: 71432
[LightGBM] [Info] Total Bins 850113
[LightGBM] [Info] Number of data points in the train set: 91109, number of used features: 3698
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.215972 -> initscore=-1.289296
[LightGBM] [Info] Start training from score -1.289296


In [5]:
# Create a DataFrame for importance
importance_df = pd.DataFrame({'feature': feature_names, 'importance': feature_importance})

# Sort by importance
importance_df = importance_df.sort_values(by='importance', ascending=False).reset_index(drop=True)

# Calculate cumulative importance
importance_df['cumulative_importance'] = importance_df['importance'].cumsum() / importance_df['importance'].sum()

# Select features contributing to 95% of cumulative importance
selected_features = importance_df[importance_df['cumulative_importance'] <= 0.95]['feature']

# Filter the train and test sets for selected features
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Retrain the model using the selected features
model_selected = lgb.LGBMClassifier(random_state=69, n_jobs=-1, force_col_wise=True)

# Train the model again on the selected features
model_selected.fit(X_train_selected, y_train)

# Make predictions and evaluate the model
y_pred_selected = model_selected.predict_proba(X_test_selected)[:, 1]
auc_roc_selected = roc_auc_score(y_test, y_pred_selected)

print(f"AUC-ROC on the selected features: {auc_roc_selected}")


[LightGBM] [Info] Number of positive: 19677, number of negative: 71432
[LightGBM] [Info] Total Bins 311277
[LightGBM] [Info] Number of data points in the train set: 91109, number of used features: 1277
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.215972 -> initscore=-1.289296
[LightGBM] [Info] Start training from score -1.289296
AUC-ROC on the selected features: 0.8723564971241229


In [6]:
X_train_selected

glucose_d1_value_range                  age  glucose_d1_min_icu_id_mean           arf_apache           age_counts  glucose_d1_max_icu_id_norm_rank  glucose_apache_d1_max_ratio  d1_hemaglobin_max_bmi_ratio       d1_glucose_max         diabete_risk  spo2_h1_max_apache_3j_diagnosis_skew  spo2_h1_max_icu_id_mean  glucose_d1_max_apache_3j_diagnosis_norm_rank        sodium_d1_std  glucose_d1_max_icu_id_diff  glucose_d1_min_icu_id_std  lactate_d1_min_icu_id_skew  creatinine_d1_min_profile_mean  glucose_h1_max_icu_id_skew  temp_h1_min_icu_id_mean  arterial_ph_h1_min_icu_id_mean  d1_wbc_max_bmi_ratio  d1_glucose_min_bmi_ratio  d1_heartrate_max_bmi_ratio  arterial_po2_h1_min_apache_3j_diagnosis_mean  temp_d1_min_icu_id_mean     sodium_d1_h1_std               icu_id     pre_icu_los_days  glucose_d1_min_icu_id_norm_rank  glucose_d1_max_apache_3j_diagnosis_diff  h1_diasbp_max_bmi_ratio  glucose_d1_min_apache_3j_diagnosis_mean  spo2_d1_max_icu_id_skew           bun_apache  \
0                       -0.33                -0.06                       -0.24                -0.17                 0.58                            -0.18                         0.36                        -0.93                -0.49                 0.43                                 -0.27                    -0.42                                         -0.49                -0.53                        0.52                       0.00                       -0.53                            1.78                        0.02                     0.62                           -1.01                  0.01                     -1.15                       -0.88                                         -0.33                     1.21                -0.77                 0.81                -0.48                            -0.40                                     0.67                    -0.53                                    -0.29                    -0.03                -0.58   
1                        1.68                -2.54                        1.49                -0.17                -2.05                             1.35                        -0.47                        -0.44                 1.82                 0.69                                 -2.59                    -1.90                                         -0.39                -0.16                       -1.67                       2.16                       -1.53                           -1.49                        0.51                     0.19                            0.05                 -0.25                      0.11                        0.07                                         -0.92                    -0.76                -0.95                -0.54                -0.63                             0.69                                     0.79                     0.33                                     0.22                     0.29                -0.68   
2                        0.25                -0.06                        0.53                -0.17                -3.84                            -0.19                         2.30                         0.22                -0.38                -2.60                                 -0.12                     0.36                                         -0.36                -0.53                        0.48                       0.74                        1.27                           -0.67                        0.53                     1.01                            0.38                  0.21                     -0.73                        1.42                                         -0.02                     1.15                 0.96                 1.46                 0.10                            -1.53                                     0.56                     0.14                                     0.31                     0.57                 0.09   
3                       -0.72                 0.67                        1

In [12]:
def find_dataframes():
    # This will check for variables that are instances of pd.DataFrame in the global scope
    return {name: obj for name, obj in globals().items() if isinstance(obj, pd.DataFrame)}

def print_memory_usage_of_dataframes():
    dataframes = find_dataframes()
    total_memory = 0
    print("Memory usage of dataframes (in GB):")
    for name, df in dataframes.items():
        mem_usage = df.memory_usage(deep=True).sum() / 1024 ** 3  # Convert bytes to gigabytes
        total_memory += mem_usage
        print(f"{name}: {mem_usage:.6f} GB")
    print(f"Total memory used by dataframes: {total_memory:.6f} GB")

print_memory_usage_of_dataframes()

Memory usage of dataframes (in GB):
X_train: 0.866253 GB
X_test: 0.371264 GB
Total memory used by dataframes: 1.237516 GB


In [8]:
X_train=X_train_selected.copy()
X_test=X_test_selected.copy()

In [13]:
X_train

glucose_d1_value_range                  age  glucose_d1_min_icu_id_mean           arf_apache           age_counts  glucose_d1_max_icu_id_norm_rank  glucose_apache_d1_max_ratio  d1_hemaglobin_max_bmi_ratio       d1_glucose_max         diabete_risk  spo2_h1_max_apache_3j_diagnosis_skew  spo2_h1_max_icu_id_mean  glucose_d1_max_apache_3j_diagnosis_norm_rank        sodium_d1_std  glucose_d1_max_icu_id_diff  glucose_d1_min_icu_id_std  lactate_d1_min_icu_id_skew  creatinine_d1_min_profile_mean  glucose_h1_max_icu_id_skew  temp_h1_min_icu_id_mean  arterial_ph_h1_min_icu_id_mean  d1_wbc_max_bmi_ratio  d1_glucose_min_bmi_ratio  d1_heartrate_max_bmi_ratio  arterial_po2_h1_min_apache_3j_diagnosis_mean  temp_d1_min_icu_id_mean     sodium_d1_h1_std               icu_id     pre_icu_los_days  glucose_d1_min_icu_id_norm_rank  glucose_d1_max_apache_3j_diagnosis_diff  h1_diasbp_max_bmi_ratio  glucose_d1_min_apache_3j_diagnosis_mean  spo2_d1_max_icu_id_skew           bun_apache  \
0                       -0.33                -0.06                       -0.24                -0.17                 0.58                            -0.18                         0.36                        -0.93                -0.49                 0.43                                 -0.27                    -0.42                                         -0.49                -0.53                        0.52                       0.00                       -0.53                            1.78                        0.02                     0.62                           -1.01                  0.01                     -1.15                       -0.88                                         -0.33                     1.21                -0.77                 0.81                -0.48                            -0.40                                     0.67                    -0.53                                    -0.29                    -0.03                -0.58   
1                        1.68                -2.54                        1.49                -0.17                -2.05                             1.35                        -0.47                        -0.44                 1.82                 0.69                                 -2.59                    -1.90                                         -0.39                -0.16                       -1.67                       2.16                       -1.53                           -1.49                        0.51                     0.19                            0.05                 -0.25                      0.11                        0.07                                         -0.92                    -0.76                -0.95                -0.54                -0.63                             0.69                                     0.79                     0.33                                     0.22                     0.29                -0.68   
2                        0.25                -0.06                        0.53                -0.17                -3.84                            -0.19                         2.30                         0.22                -0.38                -2.60                                 -0.12                     0.36                                         -0.36                -0.53                        0.48                       0.74                        1.27                           -0.67                        0.53                     1.01                            0.38                  0.21                     -0.73                        1.42                                         -0.02                     1.15                 0.96                 1.46                 0.10                            -1.53                                     0.56                     0.14                                     0.31                     0.57                 0.09   
3                       -0.72                 0.67                        1

In [17]:
model = LGBMClassifier(random_state=69, n_jobs=-1, force_col_wise=True)

boruta_selector = BorutaPy(model, random_state=69, verbose=2)

# Perform Boruta feature selection
boruta_selector.fit(X_train.values, y_train)

# Select the features based on Boruta
selected_features = X_train.columns[boruta_selector.support_].to_list()

# Filter the train and test sets for selected features
X_train_boruta = X_train[selected_features]
X_test_boruta = X_test[selected_features]

[LightGBM] [Info] Number of positive: 19677, number of negative: 71432
[LightGBM] [Info] Total Bins 622554
[LightGBM] [Info] Number of data points in the train set: 91109, number of used features: 2554
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.215972 -> initscore=-1.289296
[LightGBM] [Info] Start training from score -1.289296
Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	1277
Rejected: 	0
[LightGBM] [Info] Number of positive: 19677, number of negative: 71432
[LightGBM] [Info] Total Bins 622554
[LightGBM] [Info] Number of data points in the train set: 91109, number of used features: 2554
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.215972 -> initscore=-1.289296
[LightGBM] [Info] Start training from score -1.289296
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	1277
Rejected: 	0
[LightGBM] [Info] Number of positive: 19677, number of negative: 71432
[LightGBM] [Info] Total Bins 622554
[LightGBM] [Info] Number of data points in the train set: 91109, number of used features: 255

In [ ]:
X_train_boruta.to_csv('/Users/sakshamjain/Desktop/Projects/JAIN-WIN/X_train_boruta.csv', index=False)
X_test_boruta.to_csv('/Users/sakshamjain/Desktop/Projects/JAIN-WIN/X_test_boruta.csv', index=False)

In [18]:
model_selected = LGBMClassifier(random_state=69, n_jobs=-1, force_col_wise=True)
model_selected.fit(X_train_boruta, y_train)

# Make predictions and evaluate the model
y_pred_selected = model_selected.predict_proba(X_test_boruta)[:, 1]
auc_roc_selected = roc_auc_score(y_test, y_pred_selected)

[LightGBM] [Info] Number of positive: 19677, number of negative: 71432
[LightGBM] [Info] Total Bins 493
[LightGBM] [Info] Number of data points in the train set: 91109, number of used features: 2
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.215972 -> initscore=-1.289296
[LightGBM] [Info] Start training from score -1.289296


In [19]:
print(auc_roc_selected)

0.6968231636899214
